In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import gc
import re
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

In [ ]:
batch_size = 128
epochs = 1

max_num_features = 10
pad_size = 1
boundary_letter = -1
space_letter = 0
max_data_size = 9000000

out_path = r'.'
df = pd.read_csv(r'/kaggle/input/en-train-test/en_train.csv')
print('READ CSV')
print_data = 100000
x_data = []
y_data = pd.factorize(df['class'])
labels = y_data[1]
y_data = y_data[0]
gc.collect()

In [ ]:
count = 0
for x in df['before'].values:
    x_row = np.ones(max_num_features, dtype=int) * space_letter
    for xi, i in zip(list(str(x)), np.arange(max_num_features)):
        x_row[i] = ord(xi)
    count += 1
    if count % print_data == 0:
        print(count, 'tokens finished', end='\r')
    x_data.append(x_row)


def context_window_transform(data, pad_size):
    pre = np.zeros(max_num_features)
    pre = [pre for x in np.arange(pad_size)]
    data = pre + data + pre
    neo_data = []
    for i in np.arange(len(data) - pad_size * 2):
        row = []
        for x in data[i: i + pad_size * 2 + 1]:
            row.append([boundary_letter])
            row.append(x)
        row.append([boundary_letter])
        neo_data.append([int(x) for y in row for x in y])
    return neo_data

In [ ]:
x_data[:10]

In [ ]:
x_data = x_data[:max_data_size]
y_data = y_data[:max_data_size]
print(x_data[:10])
x_data = np.array(context_window_transform(x_data, pad_size))
gc.collect()
x_data = np.array(x_data)
y_data = np.array(y_data)

print('Total number of samples:', len(x_data))

x_train = x_data
y_train = y_data
gc.collect()

In [ ]:
x_train[:10]

In [ ]:
from keras.utils import np_utils
print(x_train.shape)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train,
                                                      test_size=0.1, random_state=2017)
num_classes = len(labels)
y_train = np_utils.to_categorical(y_train, num_classes)
y_valid = np_utils.to_categorical(y_valid, num_classes)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))
# y_train= np.reshape(y_train,(y_train.shape[0],y_train.shape[1],1))
# y_valid = np.reshape(y_valid,(y_valid.shape[0],y_valid.shape[1],1))
print(x_train.shape)
gc.collect()

In [ ]:
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=((max_num_features * 3) + 4, 1)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(labels), activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

print(x_train.shape, y_train.shape)

In [ ]:
# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_valid, y_valid))

# gc.collect()

In [ ]:
model.load_weights("/kaggle/input/entraintest/lstm_model")
model.summary()

In [ ]:
x_valid

In [ ]:
score = model.evaluate(x_valid, y_valid, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
from matplotlib import pyplot as plt

In [ ]:
# fig, ax = plt.subplots(1, 2)
# ax[0].plot(history.history['acc'], 'b')
# ax[0].set_title('Accuraccy')
# ax[1].plot(history.history['loss'], 'r')
# ax[1].set_title('Loss')
# plt.show()

In [ ]:
len(x_valid)

In [ ]:
pred = model.predict(x_valid[:10])
# print 'pred', '=====', pred
pred = [labels[np.argmax(x)] for x in pred]
pred = [labels[np.argmax(x)] for x in pred]

gc.collect()

In [ ]:
pred

In [ ]:
x_valid[:10]

In [ ]:
# x_valid = [[chr(x) for x in y[2 + max_num_features: 2 + max_num_features * 2]] for y in x_valid]
# x_valid = [''.join(x) for x in x_valid]
# x_valid = [re.sub('a+$', '', x) for x in x_valid]

In [ ]:
df_pred = pd.DataFrame(columns=['data', 'predict', 'target'])
df_pred['data'] = x_valid
df_pred['predict'] = pred
df_pred['target'] = y_valid
df_pred.to_csv(os.path.join(out_path, 'pred_lstm.csv'))

df_erros = df_pred.loc[df_pred['predict'] != df_pred['target']]
df_erros.to_csv(os.path.join(out_path, 'errors_lstm.csv'), index=False)

model.save_weights(os.path.join(out_path, 'lstm_model'))

In [ ]:
pred = model.predict(x_valid[:10])
pred